In [1]:
!pip install faker
!pip install -U bitsandbytes transformers accelerate
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.1 MB/s eta 0:00:00


In [2]:
!pip install torch==2.5.1  # Example version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

!pip uninstall -y torch torchvision torchaudio

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

!pip install --upgrade transformers accelerate

Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 836.9 kB/s 

In [4]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, pipeline
import torch
import re
import json
from faker import Faker
import random
import uuid
import pandas as pd

In [5]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [6]:
# Constants

LLAMA = "Qwen/Qwen2.5-7B-Instruct"
DATA_GEN_MODEL = LLAMA

In [7]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(DATA_GEN_MODEL, quantization_config=quantization_config, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(DATA_GEN_MODEL)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device='cuda'
)
# results = pipe(user_request, max_new_tokens=4096, do_sample=True, temperature=0.7)
# generated_text = results[0]["generated_text"]
# print(generated_text)


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [8]:

def generate_data_for_gradio(business_niche_prompt: str, num_rows: int = 1000) -> pd.DataFrame:
    """
    Generates synthetic data based on a business niche prompt using an LLM and returns a Pandas DataFrame.
    """
    print(f"\n🤖 Generating {num_rows} rows of synthetic data for: '{business_niche_prompt}'...")

    system_prompt = """System Prompt: "You are a Python Expert and Data Scientist. Your task is to write a Python script using the faker library that generates exactly [Number] rows of synthetic data based on the user's business niche.\n\nInstructions:\n1. Identify the 8-10 most relevant columns for the user's business (e.g., if 'Hospital', include patient_id, diagnosis, doctor_name).\n2. Use the faker library for names, addresses, and dates.\n3. Use random for numbers and categories.\n4. Ensure logical consistency (e.g., a 'Doctor' should not have the same ID as a 'Patient').\n5. Output ONLY the code inside a code block. No explanation text."""

    # Construct the detailed prompt for the LLM
    llm_prompt = f"""You are a Senior Python QA Engineer.
Task: Write a robust, error-free Python script to generate synthetic data for a '{business_niche_prompt}'.

STRICT RULES (Follow or the code will crash):
1. **SAFE FAKER METHODS ONLY:** You may ONLY use these specific Faker methods:
   - .name()
   - .address()
   - .email()
   - .date_this_year()
   - .text()
   - .sentence()
   - .random_number()

2. **HANDLE NICHE DATA MANUALLY:**
   - Do NOT try to use fake.genre(), fake.book_title(), fake.dish(), or any niche method. They do not exist.
   - Instead, **define a list of realistic strings** at the top of the script and use `random.choice()`.
   - Example:
     `genres = ['Fiction', 'Sci-Fi', 'Mystery']`
     `genre = random.choice(genres)`

3. **NO DUPLICATES:**
   - For IDs, strictly use: `str(uuid.uuid4())`. Do not use random integers for IDs.

4. **OUTPUT:**
   - Generate {num_rows} rows.
   - Save to 'synthetic_data.json'.
   - Output ONLY Python code in markdown blocks.
5. **DATE FORMATTING:** All date objects (e.g., from `fake.date_this_year()`) must be converted to ISO 8601 strings using `.isoformat()` before being included in the final JSON output. Example: `"order_date": fake.date_this_year().isoformat()`.
6. **CALCULATION VARIABLE ORDER:** If a variable is used in a calculation that is part of a dictionary's value, define that variable *before* the dictionary is constructed. For example, if `quantity` is generated and then used to calculate `total_price` within an `order` dictionary, define `quantity` as a separate variable first, then use it: `quantity = random.randint(1, 5)` then `"total_price": product["price"] * quantity`.
7. **FUNCTION STRUCTURE:** The generated script MUST contain two functions: `generate_synthetic_data()` which returns the list of dictionaries representing the synthetic data, and `save_to_json(data, filename)` which saves the data to a JSON file. The `generate_synthetic_data()` function MUST return a SINGLE FLAT LIST of dictionaries, where each dictionary represents one record/row and its keys are directly column names (e.g., if generating orders, each dictionary is an order, with customer and product details flattened into separate keys, not nested objects).
8. **FAKER INITIALIZATION:** Initialize the Faker object at the very beginning of your script using `fake = Faker()`.
9. **CORRECT PRODUCT LOOKUP:** When calculating `total_price` based on a `product_id`, ensure the lookup for the product's price is done correctly. For example, first select a product dictionary, then extract its `id` and `price`. Do NOT attempt to use `product_id` (a UUID string) as an integer index into a list, or perform arithmetic operations on `product_id`s directly. Example:
   `selected_product = random.choice(products)`
   `product_id = selected_product["id"]`
   `product_price = selected_product["price"]`
   `quantity = random.randint(1, 5)`
   `total_price = product_price * quantity`
   `order_data = {{ "product_id": product_id, "quantity": quantity, "total_price": total_price }}`
"""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": llm_prompt}
    ]

    try:
        # Apply chat template and call LLM
        formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        outputs = pipe(formatted_prompt, max_new_tokens=4096, do_sample=True, temperature=0.7)
        raw_response = outputs[0]["generated_text"]

        # Extract Code
        script_code = ""
        match = re.search(r"```python(.*?)```", raw_response, re.DOTALL)
        if match:
            script_code = match.group(1).strip()
        else:
            # Fallback if markdown block not found, try to extract after assistant tag
            parts = raw_response.split("<|im_start|>assistant")
            if len(parts) > 1:
                script_code = parts[-1].strip()
                if script_code.startswith("```python"):
                    script_code = script_code[len("```python"):].strip()
                if script_code.endswith("```"):
                    script_code = script_code[:-len("```")].strip()

            if not script_code:
                raise ValueError("Could not extract Python script from LLM response.")

        print("\n" + "="*40)
        print("📜 HERE IS THE GENERATED CODE:")
        print("="*40)
        print(script_code)
        print("="*40 + "\n")

        # Execute Script
        print("📝 Executing generated script...")
        exec_globals = {"Faker": Faker, "random": random, "json": json, "uuid": uuid, "pd": pd}
        exec(script_code, exec_globals)
        print("✅ Script execution complete.")

        # Call the data generation and saving functions from the executed script
        if 'generate_synthetic_data' in exec_globals and 'save_to_json' in exec_globals:
            print("💾 Generating and saving synthetic data from script...")
            synthetic_data_result = exec_globals['generate_synthetic_data']()
            exec_globals['save_to_json'](synthetic_data_result, filename='synthetic_data.json')
            print("✅ Data saved to 'synthetic_data.json'")
        else:
            raise RuntimeError("Generated script did not define 'generate_synthetic_data' or 'save_to_json'.")

        # Load JSON into DataFrame
        df = pd.read_json('synthetic_data.json')
        print("✅ Data loaded into DataFrame.")
        return df

    except Exception as e:
        print(f"❌ An error occurred during data generation: {e}")
        return pd.DataFrame() # Return empty DataFrame on error

print("Function `generate_data_for_gradio` defined.")


Function `generate_data_for_gradio` defined.


In [9]:
import gradio as gr
import os

def gradio_interface_function(business_niche_prompt: str, num_rows: int = 1000):
    """
    Generates synthetic data using the LLM and returns the DataFrame and a path to a CSV file.
    """
    print(f"\nProcessing request for: '{business_niche_prompt}' with {num_rows} rows...")
    df = generate_data_for_gradio(business_niche_prompt, num_rows)
    print(f"Debug: DataFrame empty after generation: {df.empty}") # Added print statement

    if df.empty:
        return pd.DataFrame(), None

    csv_filename = os.path.abspath('generated_data.csv')
    df.to_csv(csv_filename, index=False)
    print(f"Debug: CSV file '{csv_filename}' successfully written.")
    print(f"Data saved to temporary CSV: {csv_filename}")
    print(f"Debug: Returning CSV filename: '{csv_filename}'")
    return df, csv_filename

# Create the Gradio interface
iface = gr.Interface(
    fn=gradio_interface_function,
    inputs=[
        gr.Textbox(label='Enter business niche prompt:', placeholder='e.g., Hospital management system', lines=2),
        gr.Number(label='Number of Rows:', value=100)
    ],
    outputs=[
        gr.DataFrame(label='Generated Data'),
        gr.File(label='Download CSV')
    ],
    title='Synthetic Data Generator with LLM (Qwen/Qwen2.5-7B-Instruct)',
    description='Enter a business niche and the number of rows to generate synthetic data. The data will be displayed in a table and available for download as a CSV file.'
)

# Launch the Gradio interface
print("\nLaunching Gradio interface...")
iface.launch(share=True)


Launching Gradio interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c84c19261ee99a6065.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
